# GMRES WITH RESTART

In [1]:
import numpy as np
from numpy.linalg import qr

from functions import arnoldi, back_substitution
from preconditioner import initial_precondition, PreconditionEnum

from scipy.sparse.linalg import lsqr
from scipy.linalg import solve_triangular

In [2]:
def precon_GMRES_restarted(A, b, x0, k_max = None, restart = None, precondition = None, epsilon = 1e-12):
    """
    Restarted Generalized Minimal RESidual method for solving linear systems.
    
    - Regular mode: when restart = None the full Krylov subspace is used to solve the problem. 
        This mode is guaranteed to find a solution (if one exists), but scales poorly.
        
    - Restarted mode: when restart = N then the Krylov subspace used by the solver is rebuilt every N steps. 
        This mode is not guaranteed to find a solution for certain intial conditions, but scales well.
    
    Parameters:
    -----------
    A : numpy.ndarray
        Coefficient matrix of the linear system.
        
    b : numpy.ndarray
        Right-hand side vector of the linear system.
        
    x0 : numpy.ndarray
        Initial guess for the solution.
        
    k_max : int
        Maximum number of iterations.
        
    restart : int, optional
        Number of iterations before restart. If None, the method will not restart.
        
    epsilon : float, optional
        Tolerance for convergence.
    
    Returns:
    --------
    numpy.ndarray
        Approximate solution to the linear system.
    """
    
    n = A.shape[0]
    
    if (k_max is None):
        k_max = n
        
    elif k_max > n:
        k_max = n
    
    r0 = b - A @ x0
    M, r0, _ = initial_precondition(A, precondition, r0) # Skipping the time for now
    
    # Apply initial preconditioning
    
    p0 = np.linalg.norm(r0)
    beta = p0
    pk = p0
    k = 0
    total_k = 0
    
    while pk > epsilon*p0 and total_k < k_max:
        
        k += 1
        total_k += 1
        
        V, H = arnoldi(A, b, x0, k, precondition = precondition, M = M) # Arnoldi algorithm to generate V_{k+1} and H_{K+1, K}
        
        Q, R = qr(H, mode = 'complete')
        
        pk = abs(beta*Q[0, k]) # Compute norm of residual vector
        
        yk = back_substitution(R[:-1, :] , beta*Q[0][:-1])
    
        xk = x0 + V[:, :-1]@yk # Compute the new approximation x0 + V_{k}y
        
        if restart is not None and k == restart:
            x0 = xk
            r0 = b - A @ x0
            p0 = np.linalg.norm(r0)
            beta = p0
            pk = p0
            k = 0
            
    return xk, pk, total_k

In [3]:
import scipy.io
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import gmres
from time import time

# Load the .mtx file
A = np.array([[2,5,6],[2,3,4],[8,2,5]])

# Convert the matrix to CRS format
A = scipy.sparse.csr_matrix(A)

b = np.ones(A.shape[0])

x0 = np.zeros(b.size)

r0 = b - A @ x0

precondition = PreconditionEnum.JACOBI

M, r0, _ = initial_precondition(A, precondition, r0) # Skipping the time for now
M.toarray()

array([[2., 0., 0.],
       [0., 3., 0.],
       [0., 0., 5.]])

In [6]:
from scipy.sparse import coo_matrix

def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""
    
    nelements = 5 * N**2 - 16 * N + 16
    
    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.empty(nelements, dtype=np.float64)
    
    f = np.empty(N * N, dtype=np.float64)
    
    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  1
                f[j * N + i] = 0
                count += 1
                
            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i
                                
                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1
                
                count += 5
                                                
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

In [13]:
import scipy.io
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import gmres
from time import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from scipy.sparse.linalg import spsolve

N = 100

A, b = discretise_poisson(N)

x0 = np.zeros(b.size)

maxiter = 10000
restart = 10

start_time = time()
x, iterations = gmres(A, b, x0, restart = restart, maxiter = maxiter/restart)
residual_calculated1 = np.linalg.norm(A@x - b)
end_time = time()
print("SciPy GMRES Time:", end_time - start_time)

print(f"Calculated Scipy residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated1}")

start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = 10000, restart = restart, precondition = None)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")

SciPy GMRES Time: 0.6992659568786621
Calculated Scipy residual with Ax-b (max_iterations = 10000, restart = 10): 0.0009771182090406545
Optimized GMRES_restarted Time: 15.99144172668457
Our implementation residual with Ax-b (max_iterations = 10000, restart = 10): 1.596708356715907e-11


In [12]:
np.linalg.norm(x)

4.084541404322879